In [1]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('Desktop/2nd Sem/ML/Projects 1/sgemm_product_dataset/sgemm_product.csv')
df.head()

,MWG,NWG,KWG,MDIMC,NDIMC,MDIMA,NDIMB,KWI,VWM,VWN,STRM,STRN,SA,SB,Run1 (ms),Run2 (ms),Run3 (ms),Run4 (ms)
0,16,16,16,8,8,8,8,2,1,1,0,0,0,0,115.26,115.87,118.55,115.80
1,16,16,16,8,8,8,8,2,1,1,0,0,0,1,78.13,78.25,79.25,79.19
2,16,16,16,8,8,8,8,2,1,1,0,0,1,0,79.84,80.69,80.76,80.97
3,16,16,16,8,8,8,8,2,1,1,0,0,1,1,84.32,89.90,86.75,85.58
4,16,16,16,8,8,8,8,2,1,1,0,1,0,0,115.13,121.98,122.73,114.81


In [5]:
df = pd.DataFrame(df)
df['avg'] = df.iloc[0: ,14:].mean(axis=1)
median = df['avg'].median()
df.loc[df['avg'] >= median, 'run_class'] = 1
df.loc[df['avg'] < median, 'run_class'] = 0
df

,MWG,NWG,KWG,MDIMC,NDIMC,MDIMA,NDIMB,KWI,VWM,VWN,STRM,STRN,SA,SB,Run1 (ms),Run2 (ms),Run3 (ms),Run4 (ms),avg,run_class
0,16,16,16,8,8,8,8,2,1,1,0,0,0,0,115.26,115.87,118.55,115.80,116.3700,1.0
1,16,16,16,8,8,8,8,2,1,1,0,0,0,1,78.13,78.25,79.25,79.19,78.7050,1.0
2,16,16,16,8,8,8,8,2,1,1,0,0,1,0,79.84,80.69,80.76,80.97,80.5650,1.0
3,16,16,16,8,8,8,8,2,1,1,0,0,1,1,84.32,89.90,86.75,85.58,86.6375,1.0
4,16,16,16,8,8,8,8,2,1,1,0,1,0,0,115.13,121.98,122.73,114.81,118.6625,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241595,128,128,32,32,32,32,32,8,4,4,1,0,1,1,17.96,17.77,17.77,17.77,17.8175,0.0
241596,128,128,32,32,32,32,32,8,4,4,1,1,0,0,36.04,36.03,36.04,36.03,36.0350,0.0
241597,128,128,32,32,32,32,32,8,4,4,1,1,0,1,35.28,34.82,35.27,35.27,35.1600,0.0
241598,128,128,32,32,32,32,32,8,4,4,1,1,1,0,28.43,28.49,28.44,28.45,28.4525,0.0


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0:14],df['run_class'] , test_size=0.3, random_state=100)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std = scaler.fit_transform(X_train)

In [29]:
app=np.ones((X_train.shape[0],1))
X_std_1 = np.concatenate((app,X_std),axis=1)
beta = np.ones((1, X_train.shape[1]+1))

In [32]:
def y_hat(X_train, beta):
    y=1/(1+ np.exp(-1*(X_train@beta.T)))
    return y

In [34]:
y_train = np.reshape(np.array(y_train), (y_train.shape[0],1))

In [35]:
def costfunction(y_train, y_pred):
    cost=(-1/len(y_train))*np.sum(y_train*np.log(y_pred)+(1-y_train)*np.log(1-y_pred),axis=0)
    return cost

In [36]:
costfunction(y_train, y_hat(X_std_1, beta))

array([1.70423466])

In [37]:
def gradient_descent(X_train,y_train,beta,alpha,iterations):
    cost = np.zeros(iterations)
    
    for i in range(iterations):
        beta= beta - ((alpha/len(X_train))* np.sum(X_train*(y_hat(X_train,beta)-y_train), axis = 0))
        cost[i] = costfunction(y_train,y_hat(X_train, beta))
    return beta

In [38]:
beta_2=gradient_descent(X_std_1,y_train,beta,0.01,3000)

In [39]:
def test(X_test,beta):
    scaler = StandardScaler()
    X_test = scaler.fit_transform(X_test)

    ones = np.ones((X_test.shape[0],1))
    X_test = np.concatenate((ones, X_test), axis = 1)

    return y_hat(X_test,beta)

In [42]:
y_predicted = test(X_test,beta_2)
y_predicted

array([[0.45243814],
       [0.38335047],
       [0.41845964],
       ...,
       [0.15530215],
       [0.92207714],
       [0.34048305]])

In [44]:
log_df = pd.DataFrame(y_test.values, columns=['y_actual']) 
log_df = pd.concat([log_df, pd.DataFrame(y_predicted)], axis=1, ignore_index=True)
log_df.columns = ['y_actual', 'y_hat']
theshold_val = 0.3
log_df.loc[log_df['y_hat'] >= theshold_val, 'y_pred'] = 1
log_df.loc[log_df['y_hat'] < theshold_val, 'y_pred'] = 0

In [48]:
# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
results = confusion_matrix(log_df['y_actual'], log_df['y_pred']) 
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(log_df['y_actual'], log_df['y_pred']))
print ('Report : ')
print (classification_report(log_df['y_actual'], log_df['y_pred']) )

Confusion Matrix :
[[18534 17672]
 [ 3950 32324]]
Accuracy Score : 0.7016832229580574
Report : 
              precision    recall  f1-score   support

         0.0       0.82      0.51      0.63     36206
         1.0       0.65      0.89      0.75     36274

    accuracy                           0.70     72480
   macro avg       0.74      0.70      0.69     72480
weighted avg       0.74      0.70      0.69     72480

